In [ ]:
# These are the python modules you need to have. Please install these if you do not have them. We recommend you use Python 3.12
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


# Assignment for Summer student in GIS at Norsk e-Fuel

This is an assigment where we want to you to complete a set of three tasks. These have been designed to resemble real tasks we think you will face as a summer student at Norsk e-Fuel. The tasks should be completed in numerical order as you will need the results of one task to complete the next.

It is recommended that you have a look at the shape information file for an explanation of the column.

## Investigating wind farms in Sweden

You will for this excercise be exploring a dataset on Swedish wind farms. The data has been downloaded from the Swedish Energy Agency tool Vinbrukskollen: https://vbk.lansstyrelsen.se/.
In the investigation you will 1) Visualize wind farms on a map 2) Find the best performing wind farms in Sweden 3) Combine this in a layered map.

All the data you will need for the assignment can be found in the data folder

# Task 1 

In your first task you need to read the shape-file dataset and a map of Sweden. Fill out the missing code in each cell. The final code cell creates the map. If you get a nice looking map you have solved the task correctly.

In [ ]:
prjk_omrd_land = # Please read lst.vbk_projekteringsomraden into a GeoPandas dataframe.

url = "https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_countries.zip"  # This is a URL to fetch all the worlds country maps
world =  # Read the url into a geopandas dataframe.
sweden =  # Filter the world dataframe to select only Sweden.

In [ ]:
# You need to change the wind farm coordinate reference system to the coordinate system of the map of Sweden. Please do so.
prjk_omrd_land = # Adjust the coordinates. (Hint. https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.crs.html#geopandas.GeoDataFrame.crs)
prjk_omrd_land_aktiv = prjk_omrd_land[prjk_omrd_land["EJAKTUELL"] == "No"] # Here we filter the wind parks that have not or is not going to be built.

In [ ]:
# DON'T EDIT THIS CODE
# This is code to generate a map with the Swedish wind farms. If you have done the previous code correctly, you should get a nice map visualization.

fig, ax = plt.subplots(figsize=(8,12))

# Plot Sweden outline
sweden.plot(ax=ax, edgecolor="black", facecolor="none")

categories = prjk_omrd_land_aktiv["EL_NAMN"].unique()
colors = plt.get_cmap("Accent", len(categories))

for i, cat in enumerate(categories):
    subset = prjk_omrd_land_aktiv[prjk_omrd_land_aktiv["EL_NAMN"] == cat]  # Filter each category
    subset.plot(ax=ax, color=colors(i), label=cat)

# Create custom legend
legend_patches = [mpatches.Patch(color=colors(i), label=cat) for i, cat in enumerate(categories)]
ax.legend(handles=legend_patches, title="Bidding Zone", loc="upper right")

ax.set_axis_off()

plt.title("Wind Parks in Sweden by Bidding Zone")
plt.show()

## Task 2

You have now made a nice visulization of all wind farm projects in Sweden. Now we want to do some analysis on these wind farms. Your second task is to find the wind farms with the best performance, i.e. those producing the most electricity per turbine.

To assess the performance of a wind farm we need to calculate the farms total number of full load hours. The full load hours (abbr. FLH) is the annual electricity generation divided by the installed power capacity or max power production for the wind park. 
$$ FLH (h)= \frac{Annual\, Electricity\, Production (MWh)}{Maximum \, Power\, Output (MW)}$$

 The higher the FLH number the better. [Note, that this number can never be greater than 8760 (the total number of hours in a year).]

Once you have calculated the FLH you are to provide a list of the 20 wind parks with the highest FLH. 

The list should only show the wind park project name, the name of the municipality where it is located, the full load hours, the installed maximum capacity, and the annual electricity production.




In [ ]:
wind_turbines = # Read the Excel sheet "Land - Vindkraftverk" from the Excel file VBK_export_allman_prod.xlsx into a Pandas Dataframe.

built_wind_farmes = # Filter the wind turbines to just the turbines with "Uppført" as Status. Second, do an aggregate sum of "Maxeffect (MW)" and "Beräknad årsproduktion (GWh)" for all wind turbines by "Områdes-ID". Remember to reset the index!
built_wind_farmes = built_wind_farmes.rename(columns={"Områdes-ID" : "OMRID"})  # Here we replace the column name Områdes-ID to OMRID
swedish_windfarms = # Do an inner merge of the dataframes prjk_omrd_land_aktiv and built_wind_farms on the column OMRID.

In [ ]:
swedish_windfarms["FLH"] = # Calculat the full load hours (hint: Remember that Beräknad årsproduktion is in GWh and Maxeffekt in MW)
best_windfarms = # Sort the wind farms by FLH in descending order and filter the list to only show wind farms greater than 100 MW Maxeffekt.

# Print the dataframe and only show the columns asked for in the task description

## Task 3

You have now identified the 20 wind parks in Sweden with highest number of full load hours. We now want you to add a layer to the map in task 1, where the best wind parks are indicated on the map with a dot or point.

In [ ]:
best_windfarms["geometry"] = # Replace the geometry with a point (Hint: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.centroid.html)

In [ ]:
# DON'T EDIT THIS CODE
# This is code to generate the map visualization. If you have done the previous code correctly, you should get a nice map visualization. No need to edit this code.

fig, ax = plt.subplots(figsize=(8,10))

sweden.plot(ax=ax, edgecolor="black", facecolor="none")

categories = swedish_windfarms["EL_NAMN"].unique()  # Get unique categories
colors = plt.get_cmap("Accent", len(categories))  # Create a colormap

for i, cat in enumerate(categories):
    subset = swedish_windfarms[swedish_windfarms["EL_NAMN"] == cat]  # Filter each category
    subset.plot(ax=ax, color=colors(i), label=cat)

# Create custom legend
bidding_zone_patches = [mpatches.Patch(color=colors(i), label=cat) for i, cat in enumerate(categories)]

# **Plot best parks as scatter points**
best_windfarms.plot(
    ax=ax,
    markersize=10,
    alpha=0.5,
    label="Best Parks"  # This will show in the scatter legend
)

# **Show both legends**
scatter_legend = ax.legend( loc="upper right")  
ax.add_artist(scatter_legend)  # Keep the scatter legend

# Add bidding zones legend separately
ax.legend(handles=bidding_zone_patches, title="Bidding Zones", loc="lower right")

# Remove x and y axis
ax.set_axis_off()

plt.title("Wind Parks in Sweden")
plt.show()